Reference: https://cloud.google.com/iam/docs/quickstart-client-libraries#client-libraries-install-python

#### STEP 1 -  Install package

In [ ]:
!pip install google-cloud
!pip install google-api-python-client
!pip install oauth2client
!pip install google-cloud-iam

#### STEP 2 -  Set properties

In [ ]:
SERVICE_ACCOUNT='jupytersa'
JSON_FILE_NAME = 'sa_iam.json'
GCP_PROJECT_ID='your_project'
VERSION = 3

#### STEP 3 -  Load packages and Imports

In [ ]:
import os
import json

from google.oauth2 import service_account
import googleapiclient.discovery


#### STEP 4 -  Load API

In [ ]:
credentials = service_account.Credentials.from_service_account_file(JSON_FILE_NAME)

crm_service = googleapiclient.discovery.build(
        "cloudresourcemanager", "v1", credentials=credentials
)



#### GET/SET POLICY

In [ ]:
def get_policy(crm_service, project_id, version):
    policy = (
            crm_service.projects()
            .getIamPolicy(
                resource=project_id,
                body={"options": {"requestedPolicyVersion": version}},
            )
            .execute()
    )
    return policy

def set_policy(crm_service, project_id, policy):
    """Sets IAM policy for a project."""

    policy = (
        crm_service.projects()
        .setIamPolicy(resource=project_id, body={"policy": policy})
        .execute()
    )
    return policy



#### TESTING GET POLICY

In [ ]:
print(json.dumps(get_policy(crm_service, GCP_PROJECT_ID, VERSION),  indent=1))

#### TESTING ADD POLICY

In [ ]:
# Role to be granted.
role = "roles/logging.logWriter"
# TODO: Replace with the ID of your member in the form 'user:member@example.com'.
member = "user:member@example.com"


policy = get_policy(crm_service, GCP_PROJECT_ID, VERSION)

binding = None
for b in policy["bindings"]:
    if b["role"] == role:
        binding = b
        break
if binding is not None:
    binding["members"].append(member)
else:
    binding = {"role": role, "members": [member]}
    policy["bindings"].append(binding)

set_policy(crm_service, GCP_PROJECT_ID, policy)



#### TESTING REMOVE POLICY

In [ ]:
# Role to be granted.
role = "roles/logging.logWriter"
# TODO: Replace with the ID of your member in the form 'user:member@example.com'.
member = "user:member@example.com"


policy = get_policy(crm_service, GCP_PROJECT_ID, VERSION)

binding = next(b for b in policy["bindings"] if b["role"] == role)
if "members" in binding and member in binding["members"]:
    binding["members"].remove(member)

set_policy(crm_service, GCP_PROJECT_ID, policy)


